In [8]:
import pandas as pd
import time

In [9]:
# function to connect vertica
import vertica_python
conn_info = {'host': 'verticaz-vip.nimblestorage.com',
             'port': 5433,
             'user': 'adhoc_user1',
             'password': 'isau1',
             'database': 'dsci01',
             # autogenerated session label by default,
             #'session_label': 'some_label',
             # default throw error on invalid UTF-8 results
             #'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False
             # autocommit is off by default
             #'autocommit': True,
             # using server-side prepared statements is disabled by default
             #'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             #'connection_timeout': 10}
            }
connection = vertica_python.connect(**conn_info)
def get_data(sql_query, cnx):
    start = time.perf_counter()
    df = pd.read_sql(sql_query,cnx)
    end = time.perf_counter()
    print("Time to get the data", end - start)
    return df

In [11]:
df_20211119=get_data("""
select *
from SH_PP_reference_prep1
where "Sensors Local Timestamp"='2021-11-19 14:17:00'
""",connection)

Time to get the data 104.85219099699998


In [13]:
df_20211119.shape

(386832, 16)

In [17]:
df_20211119.head(5)

,Array SN,Sensors Local Timestamp,array_type,array_model,op_type,op_size_kb_lwr,latency_ms,num_ops,time_ops,reads,writes,readBytes,writeBytes,ntile20_model,ntile20_type,ntile20
0,AF-126092,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,0.165571,7,1159,31,785,419840,4452864,30,26,24
1,AF-126095,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.395207,488,192861,3756,908,29151744,21395456,65,55,62
2,AF-126096,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,0.226620,213,48270,487,2787,3509248,20238336,37,34,33
3,AF-126098,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,4.674625,8,37397,544,3282,10881024,32236544,97,98,98
4,AF-126102,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.410483,29,11904,5656,9281,103600128,200170496,66,57,64


In [15]:
current_model=pd.read_excel('current_production_model.xlsx')

In [16]:
current_model

,time_grain,model,io_type,op_size,coeff1,coeff2
0,minute,HFA,nsRead,0.5,0.129569,1.225750
1,minute,HFA,nsRead,1.0,0.155220,1.206110
2,minute,HFA,nsRead,2.0,0.137521,1.214983
3,minute,HFA,nsRead,4.0,0.397369,1.373327
4,minute,HFA,nsRead,8.0,0.446678,1.437339
...,...,...,...,...,...,...
193,day,AFA,write,32.0,0.000008,6.222929
194,day,AFA,write,64.0,0.000026,6.192617
195,day,AFA,write,128.0,0.000694,5.545430
196,day,AFA,write,256.0,0.008452,6.137738


In [37]:
df=pd.merge(df_20211119,current_model[current_model['time_grain']=='minute'],left_on=['array_type','op_type','op_size_kb_lwr'],right_on=['model','io_type','op_size'],how='inner')

In [38]:
df_20211119.shape,df.shape

((386832, 16), (255226, 22))

In [39]:
df.head(5)

,Array SN,Sensors Local Timestamp,array_type,array_model,op_type,op_size_kb_lwr,latency_ms,num_ops,time_ops,reads,...,writeBytes,ntile20_model,ntile20_type,ntile20,time_grain,model,io_type,op_size,coeff1,coeff2
0,AF-126092,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,0.165571,7,1159,31,...,4452864,30,26,24,minute,HFA,nsRead,4.0,0.397369,1.373327
1,AF-126095,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.395207,488,192861,3756,...,21395456,65,55,62,minute,HFA,nsRead,4.0,0.397369,1.373327
2,AF-126096,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,0.226620,213,48270,487,...,20238336,37,34,33,minute,HFA,nsRead,4.0,0.397369,1.373327
3,AF-126098,2021-11-19 14:17:00,HFA,CS200,nsRead,4.0,4.674625,8,37397,544,...,32236544,97,98,98,minute,HFA,nsRead,4.0,0.397369,1.373327
4,AF-126102,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.410483,29,11904,5656,...,200170496,66,57,64,minute,HFA,nsRead,4.0,0.397369,1.373327


In [41]:
df[df.array_model.isnull()]

,Array SN,Sensors Local Timestamp,array_type,array_model,op_type,op_size_kb_lwr,latency_ms,num_ops,time_ops,reads,...,writeBytes,ntile20_model,ntile20_type,ntile20,time_grain,model,io_type,op_size,coeff1,coeff2
1,AF-126095,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.395207,488,192861,3756,...,21395456,65,55,62,minute,HFA,nsRead,4.0,0.397369,1.373327
4,AF-126102,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.410483,29,11904,5656,...,200170496,66,57,64,minute,HFA,nsRead,4.0,0.397369,1.373327
5,AF-126103,2021-11-19 14:17:00,HFA,None,nsRead,4.0,0.757525,242,183321,945,...,278348288,85,79,84,minute,HFA,nsRead,4.0,0.397369,1.373327
6,AF-126106,2021-11-19 14:17:00,HFA,None,nsRead,4.0,1.961855,6248,12257668,19538,...,166564352,95,93,95,minute,HFA,nsRead,4.0,0.397369,1.373327
8,AF-126114,2021-11-19 14:17:00,HFA,None,nsRead,4.0,5.483085,354,1941012,3291,...,416569344,99,98,99,minute,HFA,nsRead,4.0,0.397369,1.373327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255220,AF-231160,2021-11-19 14:17:00,AFA,None,seqRead,16.0,0.135869,1609,218614,56893,...,2699423232,29,23,26,minute,AFA,seqRead,16.0,0.042572,1.790319
255221,AF-231304,2021-11-19 14:17:00,AFA,None,seqRead,16.0,0.021612,7592,164080,125503,...,473672192,5,4,5,minute,AFA,seqRead,16.0,0.042572,1.790319
255222,AF-231457,2021-11-19 14:17:00,AFA,None,seqRead,16.0,0.061167,6,367,38215,...,2328423424,16,10,14,minute,AFA,seqRead,16.0,0.042572,1.790319
255223,AF-231556,2021-11-19 14:17:00,AFA,None,seqRead,16.0,0.326688,618,201893,32212,...,4276981760,49,59,45,minute,AFA,seqRead,16.0,0.042572,1.790319
